In [9]:
from LabelEncoder import LabelEncoder
import numpy as np

In [2]:
le = LabelEncoder()

In [3]:
y_train = ["paris", "paris", "tokyo"]
le.fit(y_train)

LabelEncoder()

In [4]:
le.classes_

array(['paris', 'tokyo'], 
      dtype='|S5')

In [11]:
z= le.transform(y_train)
z

array([0, 0, 1])

In [12]:
y_test = ["paris", "paris", "tokyo" ,"amsterdam", 'honolulu']
le.transform(y_test)

array([0, 0, 1, 3, 3])

In [13]:
le.unknown_classes

array(['amsterdam', 'honolulu'], 
      dtype='|S9')

In [14]:
le.inverse_transform(np.array([0, 0, 1, -1, 0, 2, 3]))

array(['paris', 'paris', 'tokyo', 'unknown', 'paris', 'unknown', 'unknown'], dtype=object)